# Persepolise Group



## Members:

   ### *Mohsen SHAHVERDIKONDORI*
   ### *Ali NOSHAD*
   ### *Mohammad Hosein BEHZADIFARD*
   ### *Sajjad SALARI*

## Importing Libraries

In [1]:
from datasets import load_dataset
import torch
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import BertTokenizer, BertForQuestionAnswering, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from tqdm import tqdm
from transformers import AutoTokenizer
from torch import nn
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from tqdm import tqdm
from random import choice, randint
import functools
from time import time
import math

2024-05-26 18:14:50.611646: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 18:14:50.611750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 18:14:50.736945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load Dataset

In [2]:
dataset = load_dataset("eli5_category")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for eli5_category contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eli5_category
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 91772
    })
    validation1: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 5446
    })
    validation2: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 2375
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 5411
    })
})

In [4]:
dataset["train"][3]['title']

'Why is it that we calm down when we take a deep breath, hold it for a few seconds and exhale?'

## Loading Pre-trained models

In [5]:
from transformers import LongformerTokenizer, LongformerForQuestionAnswering
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from transformers import AdamW, AutoModel, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small').to("cuda:0")
param_dict = torch.load("/kaggle/input/t5-models/results (1)/QAModel_2.pth")  # has model weights, optimizer, and scheduler states
model.load_state_dict(param_dict["model"])

'''tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large").to("cuda:0")'''

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

'tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")\nmodel = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large").to("cuda:0")'

## Create the class for instance retirival and random sampling on answers

In [6]:
class ELI5Dataset(Dataset):
    def __init__(self, examples_array, extra_answer_threshold=3, min_answer_length=64, training=True, n_samples=None):
        self.data = examples_array
        self.answer_thres = extra_answer_threshold
        self.min_length = min_answer_length
        self.training = training
        self.n_samples = self.data.num_rows if n_samples is None else n_samples

    def __len__(self):
        return self.n_samples

    def make_example(self, idx):
        example = self.data[idx]
        question = example["title"]
        if self.training:
            answers = [a for i, (a, sc) in enumerate(zip(example["answers"]["text"], example["answers"]["score"]))]
            answer_tab = choice(answers).split(" ")
            start_idx = randint(0, max(0, len(answer_tab) - self.min_length))
            answer_span = " ".join(answer_tab[start_idx:])
        else:
            answer_span = example["answers"]["text"][0]
        return (question, answer_span)

    def __getitem__(self, idx):
        return self.make_example(idx % self.data.num_rows)

## Creat a batch 

In [7]:
def create_batch(qa_list, tokenizer, max_len=512, device="cuda:0"):
    q_ls = [q for q, a in qa_list]
    a_ls = [a for q, a in qa_list]
    q_toks = tokenizer(q_ls, max_length=max_len, padding="max_length", truncation=True)
    q_ids, q_mask = (
        torch.LongTensor(q_toks["input_ids"]).to(device),
        torch.LongTensor(q_toks["attention_mask"]).to(device),
    )
    a_toks = tokenizer(a_ls, max_length=max_len, padding="max_length", truncation=True)
    a_ids, a_mask = (
        torch.LongTensor(a_toks["input_ids"]).to(device),
        torch.LongTensor(a_toks["attention_mask"]).to(device),
    )
    lm_labels = a_ids[:, 1:].contiguous().clone()
    lm_labels[a_mask[:, 1:].contiguous() == 0] = -100
    model_inputs = {
        "input_ids": q_ids,
        "attention_mask": q_mask,
        "decoder_input_ids": a_ids[:, :-1].contiguous(),
        "labels": lm_labels,
    }
    return model_inputs


In [8]:
(dataset['train'])

Dataset({
    features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
    num_rows: 91772
})

## Split the data into train, validation, test

In [9]:
qar_train = ELI5Dataset(dataset['train'], training=True)
qar_valid = ELI5Dataset(dataset['validation1'], training=False)
qar_test = ELI5Dataset(dataset['test'], training=False)

In [10]:
test_sampler = RandomSampler(qar_test)

In [11]:

collate_fn_test = functools.partial(
        create_batch, tokenizer=tokenizer, max_len=512, device="cuda:0"
    )

In [12]:
print("Size of Training Set: ", len(qar_train))
print("Size of Validation Set: ", len(qar_valid))
print("Size of Test Set: ", len(qar_test))

Size of Training Set:  91772
Size of Validation Set:  5446
Size of Test Set:  5411


In [13]:
dataloader_test = DataLoader(qar_test, batch_size=4, sampler=test_sampler, collate_fn=collate_fn_test)

In [14]:
epoch_iterator_test = tqdm(dataloader_test, desc="Iteration", disable=True)

## Creating an function for doing a epoch of the training

In [15]:
def one_epoch_training(model, dataset, tokenizer, optimizer, scheduler, epoch):
    train_sampler = RandomSampler(dataset)
    collate_fn_train = functools.partial(
        create_batch, tokenizer=tokenizer, max_len=512, device="cuda:0"
    )
    dataloader_train = DataLoader(dataset, batch_size=4, sampler=train_sampler, collate_fn=collate_fn_train)
    epoch_iterator_train = tqdm(dataloader_train, desc="Iteration", disable=True)
    # accumulate loss since last print
    loc_steps = 0
    loc_loss = 0.0
    st_time = time()
    for step, batch_inputs in enumerate(epoch_iterator_train):
        if batch_inputs != 1:
            pre_loss = model(**batch_inputs)
            loss = pre_loss.loss
            loss.backward()
            # optimizer
            optimizer.step()
            scheduler.step()
            model.zero_grad()
            # some printing within the epoch
            loc_loss += loss.item()
            loc_steps += 1
            if step % 100 == 0 or step == 1:
                print(
                    "{:2d} {:5d} of {:5d} \t L: {:.10f} \t -- {:.3f}".format(
                        epoch,
                        step,
                        len(dataset) // 8,
                        loc_loss / loc_steps,
                        time() - st_time,
                    )
                )
                loc_loss = 0
                loc_steps = 0

## Creating a function for doing a epoch of validation

In [16]:
def one_epoch_evaluation(model, dataset, tokenizer, optimizer, scheduler, epoch):
    valid_sampler = RandomSampler(dataset)
    collate_fn_valid = functools.partial(
        create_batch, tokenizer=tokenizer, max_len=512, device="cuda:0"
    )
    dataloader_valid = DataLoader(dataset, batch_size=4, sampler=valid_sampler, collate_fn=collate_fn_valid)
    epoch_iterator_valid = tqdm(dataloader_valid, desc="Iteration", disable=True)
    # accumulate loss since last print
    loc_steps = 0
    loc_loss = 0.0
    st_time = time()
    with torch.no_grad():
        for step, batch_inputs in enumerate(epoch_iterator_valid):
            if batch_inputs != 1:
                pre_loss = model(**batch_inputs)
                loss = pre_loss.loss
                loc_loss += loss.item()
                loc_steps += 1
                if step % 100 == 0:
                    print(
                        "{:5d} of {:5d} \t L: {:.10f} \t -- {:.3f}".format(
                            step,
                            len(dataset) // 8,
                            loc_loss / loc_steps,
                            time() - st_time,
                        )
                    )
    
    print(
        "Total \t L: {:.3f} \t -- {:.3f}".format(
            loc_loss / loc_steps,
            time() - st_time,
        )
    )

# Function for training and validation

In [17]:
def train_question_answering(model, tokenizer, train_dataset, valid_dataset):
    optimizer = AdamW(model.parameters(), lr=2e-4, eps=1e-8)
    scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=400,
            num_training_steps=(3 + 1) * math.ceil(len(train_dataset) /8),)
    for epoch in range(2):
        one_epoch_training(model, train_dataset, tokenizer, optimizer, scheduler,epoch)
        m_save_dict = {
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
        }
        one_epoch_evaluation(model, valid_dataset, tokenizer, optimizer, scheduler, epoch)
        torch.save(m_save_dict, "{}_{}.pth".format("QAModel", epoch))

In [18]:
#train_question_answering(model, tokenizer, qar_train, qar_valid)

## Testing on the some of the data

In [19]:
# Function to generate answers for a given question
def generate_answer(question, model, tokenizer, device="cuda:0", max_len=512):
    # Prefix the question with 'question:' and tokenize it
    input_text = f"question: {question} </s>"
    input_ids = tokenizer.encode(question, return_tensors='pt').to(device)

    # Generate answer IDs
    output_ids = model.generate(input_ids, max_length=max_len, num_beams=4, early_stopping=True)

    # Decode the generated answer IDs
    generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return generated_answer

In [20]:
question = dataset["train"][6]['title']

In [21]:
dataset["train"][6]

{'q_id': '5ld008',
 'title': 'Why does lights make a "blinking" effect when looking at them from far away?',
 'selftext': '',
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dbusvko'],
  'text': ["Because of the air in between you and the light source. The air is at slightly different temperatures so it's like a heat shimmer effect you get above a fire, also the air is full of dust and other stuff we don't normally notice, but at distances it blocks a bit of the light."],
  'score': [5],
  'text_urls': [[]]},
 'title_urls': ['url'],
 'selftext_urls': ['url']}

In [22]:
generate_answer(question, model, tokenizer)

', but it\'s not a "blinking" effect, it\'s a combination of light and light. When you look at something from far away, you\'re able to see the light from far away, but it\'s not a "blinking effect" either. It\'s not a "blinking effect", it\'s a "blinking effect".'

### Generated Context Detection

In [29]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [23]:
true_ans = []
pred_ans =[]
for i in range(1000):
    true_ans.append(dataset["train"][i]['answers']['text'][0])
    pred_ans.append(generate_answer(dataset["train"][i]['title'], model, tokenizer))

In [35]:
# Function to generate text embeddings
def get_text_embedding(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    # Get the hidden states from the model
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Mean pooling to get the sentence embedding
    embeddings = outputs.last_hidden_state.mean(dim=1)
    
    return embeddings.squeeze().numpy()

In [36]:
# Combine lists into a DataFrame with labels
data = {
    "text": true_ans + pred_ans,
    "label": [0] * len(true_ans) + [1] * len(pred_ans)  # 0 for human-generated, 1 for AI-generated
}

df = pd.DataFrame(data)

In [37]:
# Apply the embedding function to the dataset
df['embedding'] = df['text'].apply(get_text_embedding)

In [38]:
# Split the data
X = np.stack(df['embedding'].values)
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [41]:
print("X train shape:", X_train.shape)
print("Y trian shape:", y_train.shape)
print("X test shape:", X_test.shape)
print("Y test shape:", y_test.shape)

X train shape: (1600, 384)
Y trian shape: (1600,)
X test shape: (400, 384)
Y test shape: (400,)


In [42]:
# Classification
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [43]:
# Predict and Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       209
           1       0.92      0.93      0.93       191

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.93       400
weighted avg       0.93      0.93      0.93       400

